# Library

In [1]:
import edward as ed
import numpy as np
import tensorflow as tf

from edward.models import Normal
from edward.stats import lognorm, norm, poisson

/opt/conda/lib/python3.5/site-packages/edward/stats/distributions.py:22: DeprecationWarning: edward.stats is deprecated. If calling rvs() from the distribution, use scipy.stats; if calling density methods from the distribution, use edward.models.
  DeprecationWarning)


In [7]:
class LatentSpaceModel():
    """
    p(x, z) = [ prod_{i=1}^N prod{j=1}^N Poi(Y_{ij}; 1/ ||z_i - z_j||) ]
              [ prod_{i=1}^N N(z_i; 0, I) ]
    """
    def __init__(self, N, K, prior_std=1.0,
                like="Poisson",
                prior="Lognormal",
                dist="euclidean"):
        self.n_vars = N * K
        self.N = N
        self.K = K
        self.prior_std = prior_std
        self.like = like
        self.prior = prior
        self.dist = dist
        
    def log_prob(self, xs, zs):
        """
        Return Scaler the log joint density log p (xs, zs)
        """
        if self.prior == "Lognormal":
            log_prior = tf.reduce_sum(lognorm.logpdf(zs["z"], self.prior_std))
        elif self.prior == "Gaussian":
            log_prior = tf.reduce_sum(norm.logpdf(zs["z"], 0.0, self.prior_std))
        else:
            raise NotImplementedError("prior not available")
            
        z = tf.reshape(zs["z"], [self.N, self.K])
        if self.dist == "euclidean":
            xp = tf.tile(tf.reduce_sum(tf.pow(z, 2), 1, keep_dims=True), [1, self.N])
            xp = xp + tf.transpose(xp) - 2 * tf.matmul(z, z, transpose_b=True)
            xp = 1.0 / tf.sqrt(xp + tf.diag(tf.zeros(self.N) + 1e3))
        elif self.dict == "cosin":
            xp = tf.matmul(z, z, transpose_b=True)
            
        if self.like == "Gaussian":
            log_lik = tf.reduce_sum(norm.logpdf(xs["x"], xp, 1.0))
        elif self.like == "Poisson":
            if not (self.dist == "euclidean" or self.prior == "Lognormal"):
                raise NotImplementedError("Rate of Poisson has to be nonnegatve")
            log_lik = tf.reduce_sum(poisson.logpmf(xs["x"], xp))
        else:
            raise NotImplementedError("likelihood not available")
            
        return log_lik + log_prior

In [3]:
ed.set_seed(42)

In [8]:
x_train = np.load("data/celegans_brain.npy")

model = LatentSpaceModel(N=x_train.shape[0], K=3, like="Poisson", prior="Gaussian")

data = {"x": x_train}
inference = ed.MAP(["z"], data, model)
inference.run(n_iter=2500)

/opt/conda/lib/python3.5/site-packages/edward/inferences/inference.py:182: DeprecationWarning: Model wrappers are deprecated. Edward is dropping support for model wrappers in future versions; use the native language instead.
  "native language instead.", DeprecationWarning)


Iteration    1 [  0%]: Loss = 66886.383
Iteration  250 [ 10%]: Loss = 38349.938
Iteration  500 [ 20%]: Loss = 37068.918
Iteration  750 [ 30%]: Loss = 36398.461
Iteration 1000 [ 40%]: Loss = 36234.676
Iteration 1250 [ 50%]: Loss = 35947.984
Iteration 1500 [ 60%]: Loss = 35882.422
Iteration 1750 [ 70%]: Loss = 35829.312
Iteration 2000 [ 80%]: Loss = 35806.688
Iteration 2250 [ 90%]: Loss = 35792.969
Iteration 2500 [100%]: Loss = 35780.410


In [ ]:
log_liks = x_post.log_prob(x_broadcasted)
log_liks = tf.reduce_sum(log_liks, 3)
log_liks = tf.reduce_mean(log_liks, 1)

cluster = tf.argmax(log_liks, 1).eval()
plt.scatter(x_train[:, 0], x_train[:, 1], c=cluster, cmap=cm.bwr)
plt.axis([-3, 3, -3, 3])
plt.title("Predicted cluster assignments")
plt.show()